In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time


In [3]:
driver = webdriver.Chrome()
driver.get('https://www.gelbeseiten.de/suche/steuerberatung/essen?umkreis=50000')
time.sleep(3)

In [4]:
driver.find_element(By.XPATH, '//*[@id="cmpwelcomebtnyes"]/a').click()

In [5]:
total = 3933
loadmorebutton = driver.find_element(By.XPATH, '//a[@id="mod-LoadMore--button"]')
profiles =driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article')

while len(profiles)!= total:
    try:
        driver.execute_script("arguments[0].scrollIntoView(false);", loadmorebutton)
        loadmorebutton.click()
        count = len(profiles)
        print(count)
        time.sleep(3)
    except:
        loadmore = False


50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
